# portfolio optimization
- dataset preparation

In [ ]:
import os
import pandas as pd

In [ ]:
# path constant
data_root = "./data/"
raw = "raw"
# manual constants

timelines = ["9008", "0924"]
timelines_alt = "0208"

In [ ]:
snp_indices = {
    "SPX": "S&P 500",  # done
    "SPLRCT": "S&P 500 Information Technology",  # done
    "SPLRCL": "S&P 500 Telecom Services",  # done
    "SPLRCM": "S&P 500 Materials",  # done
    "SPLRCREC": "S&P 500 Real Estate",  # 20020101부터 사용가능  # done
    "SPLRCS": "S&P 500 Consumer Staples",  # done
    "SPSY": "S&P 500 Financials",  # done
    "SPNY": "S&P 500 Energy",  # done
    "SPXHC": "S&P 500 Health Care",  # done
    "SPLRCD": "S&P 500 Consumer Discretionary",  # done
    "SPLRCI": "S&P 500 Industrials",  # done
    "SPLRCU": "S&P 500 Utilities",  # done
    "VIX": "CBOE Volatility Index",  # done
}  # {ticker: name} for investing.com

snp_indices_raw_data = {}

for key, value in snp_indices.items():
    temp_df_list = []
    for _each_timeline in timelines:
        if _each_timeline == "9008" and key == "SPLRCREC":
            each_timeline = timelines_alt
        else:
            each_timeline = _each_timeline
        csv_path = os.path.join(data_root, raw, f"{key}{each_timeline}.csv")    
        each_df = pd.read_csv(csv_path)
        temp_df_list.append(each_df)
    merged_df = pd.concat(temp_df_list)
    snp_indices_raw_data[key] = merged_df
    merged_df["T"] = range(len(merged_df))
    


FileNotFoundError: [Errno 2] No such file or directory: './dta/raw/SPX9008.csv'

In [2]:
test_path = "./data/raw/SPLRCD0924.csv"
test_df = pd.read_csv(test_path)

In [3]:
test_df

,Date,Price,Open,High,Low,Vol.,Change %
0,12/03/2024,"1,810.82","1,802.58","1,811.43","1,798.77",287.42M,0.13%
1,12/02/2024,"1,808.41","1,803.75","1,813.15","1,801.87",329.96M,1.06%
2,11/29/2024,"1,789.44","1,774.06","1,790.34","1,772.05",171.73M,1.11%
3,11/27/2024,"1,769.79","1,784.48","1,785.39","1,765.70",221.90M,-0.69%
4,11/26/2024,"1,782.17","1,766.98","1,783.65","1,766.08",316.84M,0.90%
...,...,...,...,...,...,...,...
4003,01/08/2009,174.78,174.21,174.78,170.79,5.29M,0.33%
4004,01/07/2009,174.21,180.54,180.54,172.95,5.30M,-3.52%
4005,01/06/2009,180.56,177.37,181.97,177.37,5.30M,1.80%
4006,01/05/2009,177.37,177.32,178.89,174.60,5.22M,0.02%
